In [1]:
import librosa
import numpy as np
from pydub import AudioSegment
import speech_recognition as sr
from pydub import AudioSegment
from pydub.playback import play
import time


def pitch_shift(audio, sample_rate, n_steps):
    return librosa.effects.pitch_shift(audio, sr=sample_rate, n_steps=n_steps)

def time_stretch(audio, rate):
    return librosa.effects.time_stretch(audio, rate=rate)

def change_amplitude(audio_segment, db_change):
    return audio_segment + db_change

def apply_equalizer(audio_segment, low_gain, mid_gain, high_gain):
    low = audio_segment.low_pass_filter(200).apply_gain(low_gain)
    mid = audio_segment.high_pass_filter(200).low_pass_filter(2000).apply_gain(mid_gain)
    high = audio_segment.high_pass_filter(2000).apply_gain(high_gain)
    return low.overlay(mid).overlay(high)

def detect_keywords(audio_file, keywords, retries=3, delay=5):
    recognizer = sr.Recognizer()
    for attempt in range(retries):
        try:
            with sr.AudioFile(audio_file) as source:
                audio = recognizer.record(source)
                transcript = recognizer.recognize_google(audio)
                print(transcript)
                detected_keywords = [kw for kw in keywords if kw in transcript.lower()]
                return detected_keywords
        except sr.RequestError as e:
            print(f"Request error: {e}. Attempt {attempt + 1}/{retries}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                return []
        except sr.UnknownValueError:
            return []

def overlay_background_sound(audio_segment, background_file, volume=-10):
    background = AudioSegment.from_file(background_file).apply_gain(volume)
    return audio_segment.overlay(background, loop=True)

def process_audio(file_path, output_path, pitch_steps, time_stretch_rate, amp_change, eq_settings, keywords_to_sounds):
    # Load audio file
    audio, sample_rate = librosa.load(file_path, sr=None)

    # Pitch Shifting
    shifted_audio = pitch_shift(audio, sample_rate, pitch_steps)

    # Time Stretching
    stretched_audio = time_stretch(shifted_audio, time_stretch_rate)

    # Convert to pydub AudioSegment for amplitude and EQ
    audio_segment = AudioSegment(
        (stretched_audio * 32767).astype(np.int16).tobytes(),
        frame_rate=sample_rate,
        sample_width=2,
        channels=1
    )

    # Amplitude Manipulation
    audio_segment = change_amplitude(audio_segment, amp_change)

    # Spectrum Manipulation (Basic EQ)
    eq_audio_segment = apply_equalizer(audio_segment, *eq_settings)
    
    # Detect keywords in the audio
    detected_keywords = detect_keywords(file_path, keywords_to_sounds.keys())

    # Load the original audio
    audio_segment = eq_audio_segment

    # Overlay background sounds based on detected keywords
    for keyword in detected_keywords:
        background_file = keywords_to_sounds[keyword]
        print(background_file)
        audio_segment = overlay_background_sound(audio_segment, background_file)

    # Export the final audio
    audio_segment.export(output_path, format="wav")




In [2]:
#file_path="/Users/user/Downloads/Mountain.mp3",
#output_path="/Users/user/Downloads/audio_output/output.wav",

keywords_to_sounds = {
    "focus": "/Users/user/Downloads/audio_output/sounds/birds-singing.mp3",
    "sitting": "/Users/user/Downloads/audio_output/sounds/heavy-rain.mp3",
}

# Example usage
process_audio(
    file_path="/Users/user/Downloads/Mountain-wav-cut.wav",
    output_path="/Users/user/Downloads/audio_output/output.wav",
    pitch_steps=2,               # Shift up 2 semitones
    time_stretch_rate=0.8,      # Slow down by 25%
    amp_change=5,                # Increase volume by 5 dB
    eq_settings=(-3, 0, 3),       # Low gain: -3 dB, Mid gain: 0 dB, High gain: +3 dB
    keywords_to_sounds=keywords_to_sounds
)

Mountain meditation this is an Equanimity practice it doesn't depend on the cultivation of concentration or open monitoring and can be used at the beginning of treatment or whenever balance or perspective is needed through Equanimity we develop the ability to stay open to suffering to meet life as it is and to be with whatever arises holding it with patience and understanding start by sitting comfortably taking a moment to ground and Center yourself
/Users/user/Downloads/audio_output/sounds/heavy-rain.mp3
